## skeleton ipynb

In [16]:
import random
import numpy as np
from game import Game, Move, Player
from copy import deepcopy
from tqdm import tqdm

Dummy game class to leave the real Game class untouched

In [2]:
class Dummy_Game(object):
    def __init__(self) -> None:
        self._board = np.ones((5, 5), dtype=np.uint8) * -1
        self.current_player_idx = 1

    def get_board(self): return self._board

    def single_move(self, board, from_pos, move, player_id):
        self._board = deepcopy(board)
        self.current_player_idx = player_id
        ok = self.__move(from_pos, move, player_id)
        return deepcopy(self._board), ok
    
    def check_winner_board(self, board):
        self._board = board
        return self.check_winner()

    def check_winner(self) -> int:
        for x in range(self._board.shape[0]):
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]): return self._board[x, 0]
        for y in range(self._board.shape[1]):
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]): return self._board[0, y]
        if self._board[0, 0] != -1 and all([self._board[x, x] for x in range(self._board.shape[0])] == self._board[0, 0]): return self._board[0, 0]
        if self._board[0, -1] != -1 and all([self._board[x, -(x + 1)] for x in range(self._board.shape[0])] == self._board[0, -1]): return self._board[0, -1]
        return -1

    def __move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        if player_id > 2: return False
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])
        acceptable = self.__take((from_pos[1], from_pos[0]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[1], from_pos[0]), slide)
            if not acceptable: self._board[(from_pos[1], from_pos[0])] = deepcopy(prev_value)
        return acceptable

    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        acceptable: bool = ((from_pos[0] == 0 and from_pos[1] < 5) or (from_pos[0] == 4 and from_pos[1] < 5) or (from_pos[1] == 0 and from_pos[0] < 5) or (from_pos[1] == 4 and from_pos[0] < 5)) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable: self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        SIDES = [(0, 0), (0, 4), (4, 0), (4, 4)]
        if from_pos not in SIDES:
            acceptable_top: bool = from_pos[0] == 0 and (slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT)
            acceptable_bottom: bool = from_pos[0] == 4 and (slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT)
            acceptable_left: bool = from_pos[1] == 0 and (slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT)
            acceptable_right: bool = from_pos[1] == 4 and (slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT)
        else:
            acceptable_top: bool = from_pos == (0, 0) and (slide == Move.BOTTOM or slide == Move.RIGHT)
            acceptable_left: bool = from_pos == (4, 0) and (slide == Move.TOP or slide == Move.RIGHT)
            acceptable_right: bool = from_pos == (0, 4) and (slide == Move.BOTTOM or slide == Move.LEFT)
            acceptable_bottom: bool = from_pos == (4, 4) and (slide == Move.TOP or slide == Move.LEFT)
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        if acceptable:
            piece = self._board[from_pos]
            if slide == Move.LEFT:
                for i in range(from_pos[1], 0, -1): self._board[(from_pos[0], i)] = self._board[(from_pos[0], i - 1)]
                self._board[(from_pos[0], 0)] = piece
            elif slide == Move.RIGHT:
                for i in range(from_pos[1], self._board.shape[1] - 1, 1): self._board[(from_pos[0], i)] = self._board[(from_pos[0], i + 1)]
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            elif slide == Move.TOP:
                for i in range(from_pos[0], 0, -1): self._board[(i, from_pos[1])] = self._board[(i - 1, from_pos[1])]
                self._board[(0, from_pos[1])] = piece
            elif slide == Move.BOTTOM:
                for i in range(from_pos[0], self._board.shape[0] - 1, 1): self._board[(i, from_pos[1])] = self._board[(i + 1, from_pos[1])]
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable

Computing all legal moves (when the board is empty)

In [3]:
border = []
for i in range(5):
    for j in range(5):
        if i == 0 or i == 4 or j == 0 or j == 4:
            border.append((i, j))
BORDER = (list(set(border)))
print(len(BORDER))

def tile_to_moves(tile):
    possible_moves = [Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT]
        
    if tile[0] == 0: possible_moves.remove(Move.LEFT)
    if tile[0] == 4: possible_moves.remove(Move.RIGHT)
    if tile[1] == 0: possible_moves.remove(Move.TOP)
    if tile[1] == 4: possible_moves.remove(Move.BOTTOM)

    return possible_moves

tile_moves = {tile: tile_to_moves(tile) for tile in BORDER}

ALL_MOVES = []
for tile in BORDER:
    possible_moves = tile_moves[tile]
    for move in possible_moves: ALL_MOVES.append((tile, move))
N_ALL = len(ALL_MOVES)

class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:

        from_pos = random.choice(BORDER)
        while game.get_board()[from_pos[1], from_pos[0]] == 1 - game.current_player_idx: from_pos = random.choice(BORDER)

        possible_moves = tile_moves[from_pos]
        
        move = random.choice(possible_moves)

        return from_pos, move

16


Checking if moves are correct and if all has been considered

In [4]:
x = np.ones((5, 5), dtype= np.uint8) * -1

count_error = 0
count_missing = 0

for i in range(5):
    for j in range(5):
        for move in [Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT]:
            if ((i, j), move) in ALL_MOVES:
                
                if not Dummy_Game().single_move(x, (i, j), move, 0)[1]:
                    print('ERROR')
                    print((i, j), move)
                    count_error += 1

            if not Dummy_Game().single_move(x, (i, j), move, 0)[1]:

                if ((i, j), move) in ALL_MOVES:
                    print('MISSING')
                    print((i, j), move)
                    count_missing += 1

print(count_error)
print(count_missing)

0
0


State could be considered as a single number if considering 0-presence and 1-presence as bits

In [5]:
import numpy as np

def state_to_board(state):
    binary_string = format(state, '050b')
    binary_array = np.array(list(map(int, binary_string))).reshape(2, 5, 5)

    board = np.zeros((5, 5), dtype=int)
    board[binary_array[0] == 1] = -1
    board[binary_array[1] == 1] = 1

    return board

def board_to_state(board):
    binary_array = np.zeros((2, 5, 5), dtype=int)
    
    binary_array[0][board == -1] = 1
    binary_array[1][board == 1] = 1

    binary_string = ''.join(map(str, binary_array.flatten()))
    return int(binary_string, 2)



rand_board = np.random.choice([-1, 0, 1], size=(5, 5), replace=True)
print('Board:')
print(rand_board)

rand_state = board_to_state(rand_board)
rand_board = state_to_board(rand_state)

print('\nState:')
print(rand_state)
print('\nBoard:')
print(state_to_board(rand_state))

Board:
[[-1  1  0 -1  1]
 [ 1  1 -1 -1  0]
 [ 0 -1  1  0 -1]
 [ 0  0  0  0  1]
 [ 1  0  0  0 -1]]

State:
640225048793136

Board:
[[-1  1  0 -1  1]
 [ 1  1 -1 -1  0]
 [ 0 -1  1  0 -1]
 [ 0  0  0  0  1]
 [ 1  0  0  0 -1]]


In [ ]:
import numpy as np

def state_to_board(state):
    binary_string = format(state, '051b')
    tmp_array = np.array(list(map(int, binary_string)))
    print(len(tmp_array))

    binary_array = tmp_array[:-1].reshape(2, 5, 5)
    next_to_move = tmp_array[-1]

    board = np.zeros((5, 5), dtype=int)
    board[binary_array[0] == 1] = -1
    board[binary_array[1] == 1] = 1

    return board, next_to_move

def board_to_state(board, next_to_move):
    binary_array = np.zeros((2, 5, 5), dtype=int)
    
    binary_array[0][board == -1] = 1
    binary_array[1][board == 1] = 1

    print(len(list(binary_array.flatten()) + [next_to_move]))
    binary_string = ''.join(map(str, list(binary_array.flatten()) + [next_to_move]))
    return int(binary_string, 2)



rand_board = np.random.choice([-1, 0, 1], size=(5, 5), replace=True)
print('starting Board:')
print(rand_board)
print(1)

rand_state = board_to_state(rand_board, 1)

print('\nState:')
print(rand_state)

print('\nreconsntructed Board:')
rec_board, next_to_move = state_to_board(rand_state)
print(rec_board)
print(next_to_move)

there are 8 simmetries total (including no simmetry) - methods to check simmetry and to rotate (from_pos, move)

In [6]:
dict_rot = {
    (Move.TOP, 1): Move.RIGHT,
    (Move.TOP, 2): Move.BOTTOM,
    (Move.TOP, 3): Move.LEFT,
    (Move.BOTTOM, 1): Move.LEFT,
    (Move.BOTTOM, 2): Move.TOP,
    (Move.BOTTOM, 3): Move.RIGHT,
    (Move.LEFT, 1): Move.TOP,
    (Move.LEFT, 2): Move.RIGHT,
    (Move.LEFT, 3): Move.BOTTOM,
    (Move.RIGHT, 1): Move.BOTTOM,
    (Move.RIGHT, 2): Move.LEFT,
    (Move.RIGHT, 3): Move.TOP,
}

dict_flip = {
    Move.TOP: Move.TOP,
    Move.BOTTOM: Move.BOTTOM,
    Move.LEFT: Move.RIGHT,
    Move.RIGHT: Move.LEFT,
}

#rot_orario: (3, 4) -> (4, 1) -> (1, 0) -> (0, 3) -> (3, 4)
#: (xi, yi) -> (yi, 4 - xi)
#rot_anti_orario: (3, 4) -> (0, 3) -> (1, 0) -> (4, 1) -> (3, 4)
#: (xi, yi) -> (4 - yi, xi)

def rot(n_rot):
    def rot_n(from_pos, move):
        for _ in range(n_rot):
            from_pos = 4 - from_pos[1], from_pos[0]
        return from_pos, dict_rot[(move, n_rot)]
    return rot_n

def flip(from_pos, move):
    from_pos = 4 - from_pos[0], from_pos[1]
    return from_pos, dict_flip[move]

def flip_rot(n_rot):
    def flip_rot_n(from_pos, move):
        from_pos, move = rot(n_rot)(from_pos, move)
        return flip(from_pos, move)
    return flip_rot_n

rot1 = rot(1)
rot2 = rot(2)
rot3 = rot(3)
flip_rot1 = flip_rot(1)
flip_rot2 = flip_rot(2)
flip_rot3 = flip_rot(3)

verse_simmetries = [
    rot3,
    rot2,
    rot1,
    flip,
    flip_rot3,
    flip_rot2,
    flip_rot1,
]

inverse_simmetries = [
    rot1,
    rot2,
    rot3,
    flip,
    flip_rot1,
    flip_rot2,
    flip_rot3,
]

def check_simmetries(board, state_list):

    base_state = tuple(board.flatten())
    if base_state in state_list: return base_state, None

    R1 = np.rot90(board)
    base_state = tuple(R1.flatten())
    if base_state in state_list: return base_state, 0

    R2 = np.rot90(R1)
    base_state = tuple(R2.flatten())
    if base_state in state_list: return base_state, 1

    R3 = np.rot90(R2)
    base_state = tuple(R3.flatten())
    if base_state in state_list: return base_state, 2
    
    F = np.fliplr(board)
    base_state = tuple(F.flatten())
    if base_state in state_list: return base_state, 3
    
    FR1 = np.rot90(F)
    base_state = tuple(FR1.flatten())
    if base_state in state_list: return base_state, 4
    
    FR2 = np.rot90(FR1)
    base_state = tuple(FR2.flatten())
    if base_state in state_list: return base_state, 5
    
    FR3 = np.rot90(FR2)
    base_state = tuple(FR3.flatten())
    if base_state in state_list: return base_state, 6
    
    return None

def check_simmetries(board, next_to_move, state_list):

    base_state = tuple(list(board.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, None

    R1 = np.rot90(board)
    base_state = tuple(list(R1.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 0

    R2 = np.rot90(R1)
    base_state = tuple(list(R2.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 1

    R3 = np.rot90(R2)
    base_state = tuple(list(R3.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 2
    
    F = np.fliplr(board)
    base_state = tuple(list(F.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 3
    
    FR1 = np.rot90(F)
    base_state = tuple(list(FR1.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 4
    
    FR2 = np.rot90(FR1)
    base_state = tuple(list(FR2.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 5
    
    FR3 = np.rot90(FR2)
    base_state = tuple(list(FR3.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 6
    
    return None

MOVES_SIMMETRIES = {} #(id_move, id_simmetry) -> id_move

for id_move in range(len(ALL_MOVES)):
    from_pos, move = ALL_MOVES[id_move]

    for id_simmetry in range(len(inverse_simmetries)):

        idx = None
        for i in range(len(ALL_MOVES)):
            if ALL_MOVES[i] == inverse_simmetries[id_simmetry](from_pos, move):
                idx = i
                break
        
        MOVES_SIMMETRIES[(id_move, id_simmetry)] = i

print(len(MOVES_SIMMETRIES))
print(len(ALL_MOVES) * 7)
c = 10
for k, v in MOVES_SIMMETRIES.items():
    print((k, v))
    c -= 1
    if c == 0: break

308
308
((0, 0), 31)
((0, 1), 18)
((0, 2), 27)
((0, 3), 41)
((0, 4), 39)
((0, 5), 21)
((0, 6), 14)
((1, 0), 30)
((1, 1), 17)
((1, 2), 28)


checking if simmetries works

In [7]:
move_to_check = 14

x = np.ones((5, 5)) * -1
x[0, 1] = 0
x[2, 4] = 0
print('x')
print(x)
print(ALL_MOVES[move_to_check])
print(Dummy_Game().single_move(x, ALL_MOVES[move_to_check][0], ALL_MOVES[move_to_check][1], 0)[0])
print('----------------------------')
print('----------------------------')
state = tuple(x.flatten())
di = [state]

y = np.rot90(x, k= 1)
#for y in [np.rot90(x, k= 1), np.rot90(x, k= 2), np.rot90(x, k= 3), np.fliplr(x)]:
#for y in [np.fliplr(x)]:
for y in [np.rot90(np.fliplr(x), k= 1)]:
    print('y')
    print(y)
    print('----------------------------')
    print('----------------------------')

    b, s = check_simmetries(y, di) # at the start of make_move, to get the state and the simmetry_id
    print(np.array(b).reshape(5, 5))
    print(s)
    print('----------------------------')
    print('----------------------------')
    id_move = MOVES_SIMMETRIES[(move_to_check, s)] # at the end, when the move for the state in memory is decided and have to be simmetrized
    print(id_move)
    print(ALL_MOVES[id_move])
    print(Dummy_Game().single_move(y, ALL_MOVES[id_move][0], ALL_MOVES[id_move][1], 0)[0])

    print('=================================')
    print('=================================')

x
[[-1.  0. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
((3, 4), <Move.RIGHT: 3>)
[[-1.  0. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]]
----------------------------
----------------------------
y
[[-1. -1. -1. -1. -1.]
 [ 0. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1.  0. -1. -1.]]
----------------------------
----------------------------
[[-1.  0. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]]
4
----------------------------
----------------------------
18
((4, 3), <Move.BOTTOM: 1>)
[[-1. -1. -1. -1. -1.]
 [ 0. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1.  0. -1.  0.]]
